In [ ]:
#https://aminer.org/aminernetwork
import re
import csv
import json
#import ijson
import bisect
#import gensim
import nltk
import _thread
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import scipy.interpolate as interpolate
from operator import itemgetter
from scipy import stats
from itertools import groupby
from collections import Counter
#from gensim.models import LdaModel
from stop_words import get_stop_words
from nltk import pos_tag
from IPython.display import clear_output
from multiprocessing.dummy import Pool as ThreadPool
from nltk.corpus import stopwords as nltk_stop_words, wordnet as wn
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from scipy.interpolate import spline
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
folder_path = 'D:/MS CS/RS DATA/dataset/'
print("Success")

### -> Just Initialize variables

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    STEXT = 'Note:- If you want to '
    ETEXT = ', then run next cell otherwise ignore it...'
    EETEXT = ' otherwise ignore it...'

clear_output()
#default number of papers
number_of_papers = 0

#just file name
file_number = 4

# path where files reside
folder_path = 'D:/MS CS/RS DATA/dataset/backup/'
#folder_path = 'E:/MS CS/RS DATA/dblp-ref/'

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')
# Bring in the default English NLTK stop words
en_stop += nltk_stop_words.words('english')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

print("All variables have initialized...")
print(bcolors.STEXT+bcolors.HEADER+"perform pre-precessing on new dataset"+bcolors.ENDC+bcolors.ETEXT)

In [ ]:
citation_of_paper = pd.read_csv(folder_path+'cite.csv', sep=',', names=['pid', 'citation'])
rs_corpus = []
with open(folder_path+'rs-corpus.json','r', encoding="utf8") as f:
    for doc in f:
        rs_corpus.append(json.loads(doc))
    f.close()
print("dataset loaded...!!")

In [ ]:
for doc in rs_corpus:
    citation = list(citation_of_paper[citation_of_paper['pid']==doc['id']]['citation'])
    if len(citation)>0:
        doc['citation'] = int(citation[0])
    else:
        doc['citation'] = 0
pd.DataFrame(rs_corpus).to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print("success")

## Operations on papers corpus
#### -> convert text to JSON file (RS and VS corpus)
#### -> tokenization
#### -> stop words removal

In [ ]:
# list for tokenized documents in loop
#file_rs = open(folder_path+'rs-corpus.json', 'w+', encoding="utf8")
#file_vs = open(folder_path+'vs-corpus.json', 'w+', encoding="utf8")

def topic_preprocessing(data_topic):
    # clean and tokenize document string
    raw = (data_topic.lower().strip(':-;?!.,@')).strip()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if i not in en_stop]

    # steme words from stopped_tokens
    stemmed_tokens = []
    for i in stopped_tokens:
        stm = p_stemmer.stem(i.strip())
        if len(stm)>1 and re.match('^[a-zA-Z0-9_]+$', stm) and not stm.isdigit():
            tag = wn.synsets(stm)
            if not tag:
                stemmed_tokens.append(stm)
            if tag:
                pos = tag[0].pos()
                if pos != 'v':
                    stemmed_tokens.append(stm)
            
    return stemmed_tokens

def add_doc_to_rs(rs_data):
    #clear_output()
    #print(rs_data['title'])
    #print(rs_data['abstract'])
    #rs_data['title'] = topic_preprocessing(rs_data['title'])
    #rs_data['abstract'] = topic_preprocessing(rs_data['abstract'])
    file_rs.write(json.dumps(rs_data)+'\n')
    
def add_doc_to_vs(vs_data):
    file_vs.write(json.dumps(vs_data)+'\n')

def init_param():
    global data
    data = {}
    global title
    title = ''
    global abstract
    abstract = ''
    global pub_year
    pub_year = 1000
    global references
    references = []
    
if __name__ == "__main__":
    
    doc_set_rs = []
    doc_set_vs = []
    #pool = ThreadPool(4)
    count = 0
    with open(folder_path+'AMiner-Paper.txt', 'r', encoding="utf8") as f:
        init_param()
        for line in f:
            line = line.strip()
            if line.startswith( '#index' ):
                data['id'] = int((line.split('#index')[1]).strip())
            elif line.startswith( '#*' ):
                title = (line.split('#*')[1]).strip()
            elif line.startswith( '#@' ):
                data['authors'] = [a.strip() for a in (line.split('#@')[1]).split(';')]
            elif line.startswith( '#o' ):
                data['affiliations'] = [a.strip() for a in (line.split('#o')[1]).split(';')]
            elif line.startswith( '#t' ):
                year = (line.split('#t')[1]).strip()
                if not year or year == "":
                    data['year'] = 0
                else:
                    pub_year = int(year)
                    data['year'] = pub_year
            elif line.startswith( '#c' ):
                data['venue'] = (line.split('#c')[1]).strip()
            elif line.startswith( '#%' ):
                references.append(int((line.split('#%')[1]).strip()))
            elif line.startswith( '#!' ):
                abstract = (line.split('#!')[1]).strip()
            else:
                count += 1
                if count % 100000 == 0:
                    print(count, "documents selected...")
                if not line or line == "":
                    if not data.keys():
                        init_param()
                    else:
                        data['references'] = references
                        if pub_year > 2009:
                            if title:
                                data['title'] = title
                                doc_set_vs.append(data)
                        elif pub_year > 2004:
                            if title:
                                data['title'] = title
                                doc_set_rs.append(data)
                        init_param()
        
        clear_output()                        
        print(count, "documents selected...")                        
        print("Data preprocessing has been started...")
        # and return the results
        #for data in doc_set_rs:
            #add_doc_to_rs(data)
        #pool.map(add_doc_to_rs, doc_set_rs)
        #pool.map(add_doc_to_vs, doc_set_vs)
        pd.DataFrame(doc_set_vs).to_json(folder_path+'doc_set_vs.json', orient='records', lines=True)
        pd.DataFrame(doc_set_rs).to_json(folder_path+'doc_set_rs.json', orient='records', lines=True)
        # close the pool and wait for the work to finish 
        #pool.close()
        #pool.join()
        #file_rs.close()
        #file_vs.close()
        #clear_output()
        print("Data preprocessing has been done and saved...")
        print(bcolors.STEXT+bcolors.HEADER+"pre-process authors doc-set"+bcolors.ENDC+bcolors.ETEXT)

In [ ]:
doc_set_rs[doc_set_rs.id.isin(author_to_paper.pid.tolist())].to_json(folder_path+'doc_set_rs2.json', orient='records', lines=True)
doc_set_vs[doc_set_vs.id.isin(vs_corpus_avg.id.tolist())].to_json(folder_path+'doc_set_vs2.json', orient='records', lines=True)

In [ ]:
doc_set_rs = pd.read_json(folder_path+'doc_set_rs2.json', orient='records', encoding='utf8', lines=True)
doc_set_vs = pd.read_json(folder_path+'doc_set_vs2.json', orient='records', encoding='utf8', lines=True)
#vs_corpus_avg = pd.read_json(folder_path+'vs_corpus_avg.json', orient='records', encoding='utf8', lines=True)
#author_to_paper = pd.read_json(folder_path+'author_to_paper.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
def topic_preprocessing(data_topic):
    # clean and tokenize document string
    raw = (data_topic.lower().strip(':-;?!.,@')).strip()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens     
    return [i for i in tokens if i not in en_stop and len(i)>1 and re.match('^[a-zA-Z0-9_]+$', i) and not i.isdigit()]

if __name__ == "__main__":
    
    set_rs = []
    set_vs = []
    for k, v in doc_set_rs.iterrows():
        v.title = topic_preprocessing(v.title)
        set_rs.append(v)
    
    for k, v in doc_set_vs.iterrows():
        v.title = topic_preprocessing(v.title)
        set_vs.append(v)
        
    pd.DataFrame(set_vs).to_json(folder_path+'set_vs.json', orient='records', lines=True)
    pd.DataFrame(set_rs).to_json(folder_path+'set_rs.json', orient='records', lines=True)
    print("success")

In [ ]:
file_vs = open(folder_path+'vs-corpus-reff.json', 'w+', encoding="utf8")

def topic_preprocessing(data_topic):
    # clean and tokenize document string
    raw = (data_topic.lower().strip(':-;?!.,@')).strip()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if i not in en_stop]

    # steme words from stopped_tokens
    stemmed_tokens = []
    for i in stopped_tokens:
        stm = p_stemmer.stem(i.strip())
        if len(stm)>1 and re.match('^[a-zA-Z0-9_]+$', stm) and not stm.isdigit():
            tag = wn.synsets(stm)
            if not tag:
                stemmed_tokens.append(stm)
            if tag:
                pos = tag[0].pos()
                if pos != 'v':
                    stemmed_tokens.append(stm)
            
    return stemmed_tokens
    
def add_doc_to_vs(vs_data):
    #clear_output()
    #print(vs_data['title'])
    #vs_data['title'] = topic_preprocessing(vs_data['title'])
    file_vs.write(json.dumps(vs_data)+'\n')

def init_param():
    global data
    data = {}
    global title
    title = ''
    global abstract
    abstract = ''
    global pub_year
    pub_year = 1000
    global references
    references = []
    
if __name__ == "__main__":
    
    doc_set_rs = []
    doc_set_vs = []
    #pool = ThreadPool(4)
    count = 0
    with open(folder_path+'backup/AMiner-Paper.txt', 'r', encoding="utf8") as f:
        init_param()
        for line in f:
            line = line.strip()
            if line.startswith( '#index' ):
                data['id'] = int((line.split('#index')[1]).strip())
            elif line.startswith( '#*' ):
                title = (line.split('#*')[1]).strip()
            elif line.startswith( '#@' ):
                temp = 1
                #data['authors'] = [a.strip() for a in (line.split('#@')[1]).split(';')]
            elif line.startswith( '#o' ):
                temp = 1
                #data['affiliations'] = [a.strip() for a in (line.split('#o')[1]).split(';')]
            elif line.startswith( '#t' ):
                year = (line.split('#t')[1]).strip()
                if not year or year == "":
                    data['year'] = 0
                else:
                    pub_year = int(year)
                    data['year'] = pub_year
            elif line.startswith( '#c' ):
                temp = 1
                #data['venue'] = (line.split('#c')[1]).strip()
            elif line.startswith( '#%' ):
                references.append(int((line.split('#%')[1]).strip()))
            elif line.startswith( '#!' ):
                temp = 1
                #abstract = (line.split('#!')[1]).strip()
            else:
                count += 1
                if count % 100000 == 0:
                    print(count, "documents selected...")
                if not line or line == "":
                    if not data.keys():
                        init_param()
                    else:
                        if (pub_year > 2009) and len(references)>0:
                            data['references'] = references
                            #data['title'] = title
                            doc_set_vs.append(data)
                        init_param()
                        
        
        clear_output()                        
        print(count, "documents selected...")                        
        print("Data preprocessing has been started...")
        # and return the results
        for data in doc_set_vs:
            add_doc_to_vs(data)
        file_vs.close()
        clear_output()
        print("Data preprocessing has been done and saved...")
        print(bcolors.STEXT+bcolors.HEADER+"pre-process authors doc-set"+bcolors.ENDC+bcolors.ETEXT)

In [ ]:
id = 0
with open(folder_path+'backup/AMiner-Paper.txt', 'r', encoding="utf8") as f:

    for line in f:
        line = line.strip()
        if line.startswith( '#index' ):
            id = int((line.split('#index')[1]).strip())
        elif line.startswith( '#*' ):
            title = (line.split('#*')[1]).strip()
            if id == 2077796:
                print(title)
                break

In [ ]:
rs_corpus = []
with open(folder_path+'rs-corpus.json','r', encoding="utf8") as f:
    for doc in f:
        rs_corpus.append(json.loads(doc))
    f.close()

## Operations on Author-2-Paper
#### -> Links of senior authors removed

In [ ]:
senior_author_pid = []
with open(folder_path+'vs-corpus.json','r', encoding="utf8") as f:
    for doc in f:
        senior_author_pid.append(json.loads(doc)['id'])
    f.close()
    
author_to_paper = pd.read_csv(folder_path+'AMiner-Author2Paper.txt', sep='\t', names=['aid', 'pid', 'pos'])
print("started")
senior_author = author_to_paper[author_to_paper['pid'].isin(senior_author_pid)]
senior_author_pid = list(set(senior_author['pid']))
author_to_paper = author_to_paper[~author_to_paper['pid'].isin(senior_author_pid)]
author_to_paper.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
pd.DataFrame(senior_author, columns=['aid', 'pid', 'pos']).to_csv(folder_path+'senior_author.csv', sep=',', encoding='utf-8', index=False)
print("success")
print(len(author_to_paper))

In [ ]:
author_to_paper = pd.read_csv(folder_path+'AMiner-Author2Paper.txt', sep='\t', names=['aid', 'pid', 'pos'])
paper_id = author_to_paper[(author_to_paper.aid.isin(author.aid)) & (author_to_paper.pos == 1)].pid
author_to_paper = author_to_paper[author_to_paper.pid.isin(paper_id)]
author_to_paper.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
rs_corpus[rs_corpus.id.isin(paper_id)]
rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
#author_to_paper[(author_to_paper.aid==481344) & (author_to_paper.pos==1)]
#author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
senior_author = pd.read_csv(folder_path+'senior_author.csv', sep=',', names=['aid', 'pid', 'pos'])

In [ ]:
vs_author_to_paper = pd.read_json(folder_path+'vs-author-2-paper.json', orient='records', encoding='utf8', lines=True)
vs_corpus = pd.read_json(folder_path+'vs-corpus.json', orient='records', encoding='utf8', lines=True)
grouped_vs_corpus = vs_corpus.groupby("year")
print("success")

In [ ]:
pid = []
for year, data in grouped_vs_corpus:
    for k, v in vs_author_to_paper[(vs_author_to_paper.pid.isin(data.id)) & (vs_author_to_paper.pos==1)].groupby("aid"):
        if v.pid.count()<3:
            pid.append(v.pid.tolist())
            
from itertools import chain
pid = list(chain.from_iterable(pid))
print(len(pid))

## VS Operation

In [ ]:
vs_corpus_reff = pd.read_json(folder_path+'vs-corpus-reff.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
grouped_vs_corpus_reff = vs_corpus_reff.groupby("year")
print("success")

In [ ]:
year_ref_set = []
for k, v in grouped_vs_corpus_reff:
    
    year_ref = {}
    ref = []
    for reff in v.references:
        ref.append(len(reff))
    year_ref['year'] = k
    year_ref['references'] = ref
    year_ref["total_paper"] = v.references.count()
    year_ref_set.append(year_ref)
print("success")

In [ ]:
import statistics

reff_set = []
for v in year_ref_set:
    
    data = {}
    data["year"] = v['year']
    data["avg"] = (sum(v['references'])/len(v['references']))
    data["stdev"] = statistics.stdev(v['references'])
    reff_set.append(data)
    
    print("year:", data["year"])
    print("avg", data["avg"])
    print("stdev", data["stdev"])
    print("-------------------")

In [ ]:
stdev_pid =[]
avg_pid = []
for row in reff_set:
    for k, paper in vs_corpus_reff[(vs_corpus_reff.year == row['year'])].iterrows():
        
        if len(paper.references)<row['stdev']:
            stdev_pid.append(paper.id)
        if paper.year == 2014:
            if len(paper.references)<25:
                avg_pid.append(paper.id)
        elif len(paper.references)<33:
            avg_pid.append(paper.id)
            
print("stdev_pid", len(stdev_pid))
print("avg_pid", len(avg_pid))
print("success")

g_dummy = vs_corpus_reff[~vs_corpus_reff.id.isin(avg_pid)]
#g_dummy = g_dummy[~g_dummy.id.isin(avg_pid)]

for year, v in g_dummy.groupby('year'):
    print(year)
    print(len(v))
    print("---------------------------------")

In [ ]:
vs_similarity_corpus = pd.read_csv(folder_path+'vs-topic-paper-similarity-corpus.csv', names=['tid', 'pid', 'year', 'sim'])
for year, v in vs_similarity_corpus.groupby('year'):
    print(year)
    print(len(v))
    print("---------------------------------")

In [ ]:
vs_corpus_reff = vs_corpus_reff[~vs_corpus_reff.id.isin(avg_pid)]
g_dummy = vs_corpus_reff[~vs_corpus_reff.id.isin(avg_pid)]

for year, v in g_dummy.groupby('year'):
    print(year)
    print(len(v))
    print("---------------------------------")

In [ ]:
vs_corpus = pd.read_json(folder_path+'vs_corpus_avg.json', orient='records', encoding='utf8', lines=True)
vs_corpus = vs_corpus[vs_corpus.id.isin(vs_corpus_reff.id.tolist())]
vs_corpus.to_json(folder_path+'vs_corpus_avg.json', orient='records', lines=True)
print("success")

In [ ]:
vs_similarity_corpus = pd.read_csv(folder_path+'complete_vs_similarity_corpus.csv', names=['tid', 'pid', 'year', 'sim'])
vs_similarity_corpus = vs_similarity_corpus[vs_similarity_corpus.pid.isin(vs_corpus.id)]
vs_similarity_corpus.to_csv(folder_path+'vs_similarity_corpus.csv', sep=',', encoding='utf-8', index=False)
print("success")

## Operations on author corpus
#### -> Just convert text - to - JSON file
#### -> Select authors (Juniors & Seniors), who publish paper in RS
#### -> Remove authors, who havn't affiliation

In [ ]:
data = {}
author_corpus = []
with open(folder_path+'AMiner-Author.txt', 'r', encoding="utf8") as f:

    for line in f:
        line = line.strip()
        if line.startswith( '#index' ):
            data['aid'] = int((line.split('#index')[1]).strip())
        elif line.startswith( '#n' ):
            data['authors'] = [a.strip() for a in (line.split('#n')[1]).split(';')]
        elif line.startswith( '#a' ):
            data['affiliations'] = [a.strip() for a in (line.split('#a')[1]).split(';')]
        elif line.startswith( '#pc' ):
            data['n_paper'] = int((line.split('#pc')[1]).strip())
        elif line.startswith( '#cn' ):
            data['n_citation'] = int((line.split('#cn')[1]).strip())
        elif line.startswith( '#hi' ):
            data['h_index'] = int((line.split('#hi')[1]).strip())
        elif line.startswith( '#pi' ):
            data['pa_index'] = float((line.split('#pi')[1]).strip())
        elif line.startswith( '#upi' ):
            data['p_index'] = float((line.split('#upi')[1]).strip())
        elif line.startswith( '#t' ):
            data['area'] = [a.strip() for a in (line.split('#t')[1]).split(';')]
        else:
            if not line or line == "":
                if not data.keys():
                    data = {}
                else:
                    author_corpus.append(data)
                    data = {}

    f.close()
    print("processing successfully started")
    author_corpus = pd.DataFrame(author_corpus)
    author_corpus = author_corpus[author_corpus['aid'].isin(author_to_paper['aid'])]
    #author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
    print("junior authors", len(author_corpus))
    print("starts")
    author_with_affiliation = pd.DataFrame([row for index, row in author_corpus.iterrows() if len(row['affiliations']) == 1 and row['affiliations'][0] != ''])
    aid_without_affiliation = author_corpus[~(author_corpus['aid']).isin(author_with_affiliation['aid'])]['aid']
    author_with_affiliation = author_with_affiliation[~(author_with_affiliation['aid']).isin(senior_author['aid'])]
    print("author_with_affiliation starts")
    author_with_affiliation.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
    aid_without_affiliation.to_csv(folder_path+'aid_without_affi.csv', sep=',', encoding='utf-8', index=False)
    print("remove authors with affic", len(author_with_affiliation))
    print("remove authors without affic", len(aid_without_affiliation))
    print("authors processing successfully done and saved")

#### -> Remove authors link and papers, who havn't affiliation

In [ ]:
#aid_without_affi = pd.read_csv(folder_path+'aid_without_affi.csv', sep=',', names=['aid_without_affi'])
#author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
#rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
aid_without_affiction = list(aid_without_affi['aid_without_affi'])
print("initial rs_corpus", len(rs_corpus))
print("author_to_paper", len(author_to_paper))
print("started")
rs_corpus_pid = list(set(author_to_paper.loc[(author_to_paper['aid'].isin(aid_without_affiction)) & (author_to_paper["pos"] == 1)]['pid'].values))
rs_corpus = rs_corpus[~rs_corpus['id'].isin(rs_corpus_pid)]
author_to_paper = author_to_paper[~author_to_paper['pid'].isin(rs_corpus_pid)]
author_to_paper.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print("after romoving author_to_paper", len(author_to_paper))
print("after romoving rs_corpus", len(rs_corpus))
print("success")

### Operations on Author-2-Paper corpus
#### -> count Authors papers (papers>2 or papers=<2)

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_to_paper_groupby = author_to_paper.groupby("aid")

author_less_than_3_papers = []
for index, row in author_to_paper_groupby:
    data = row[(row['pos'] == 1) & (len(row['pos'] == 1)<3)]
    if len(data['aid'])>0:
        for index, rec in data.iterrows():
            author_less_than_3_papers.append(list(rec))
author_less_than_3_papers = pd.DataFrame(author_less_than_3_papers, columns = ['aid','pid','pos'])
author_less_than_3_papers.to_csv(folder_path+'author_less_than_3_papers.csv', sep=',', encoding='utf-8', index=False)
print("success")

#### -> remove authors and links, who have less than 2 papers

In [ ]:
#author_less_than_3_papers = pd.read_csv(folder_path+'author_less_than_3_papers.csv', sep=',', header=0, index_col=False)
#author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
rs_aid = list(set(author_less_than_3_papers['aid']))
rs_pid = list(set(author_less_than_3_papers['pid']))
author_corpus = author_corpus[~author_corpus['aid'].isin(rs_aid)]
author_to_paper = author_to_paper[~author_to_paper['pid'].isin(rs_pid)]
rs_corpus = rs_corpus[~rs_corpus['id'].isin(rs_pid)]
author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
author_to_paper.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print("success")
print(len(author_corpus))
print(len(author_to_paper))
print(len(rs_corpus))

In [ ]:
paper_id = author_to_paper[author_to_paper.pos==1]['pid'].values
author_id = author_to_paper[author_to_paper.pos==1]['aid'].values

In [ ]:
rs_corpus = rs_corpus[rs_corpus.id.isin(paper_id)]
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
paper_id = author_to_paper[author_to_paper.pos==1]['pid'].values
author_id = author_to_paper[author_to_paper.pos==1]['aid'].values
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)

In [ ]:
print(len(author_to_paper))
author_to_paper = author_to_paper[author_to_paper['pid'].isin(rs_corpus['id'])]
author_to_paper.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
print(len(author_to_paper))

In [ ]:
print(len(author_corpus))
print(len(rs_corpus))
author_corpus = author_corpus[author_corpus['aid'].isin(author_id)]
rs_corpus = rs_corpus[rs_corpus['id'].isin(paper_id)]
author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print(len(author_corpus))
print(len(rs_corpus))

In [ ]:
aid = list(set(author_corpus['aid']))
author_pid = author_to_paper[author_to_paper['aid'].isin(aid)]['pid'].values
author_pid = list(set(author_pid))
authors = author_to_paper[author_to_paper['pid'].isin(author_pid)]
authors.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
rscorpus = rs_corpus[rs_corpus['id'].isin(author_pid)]
rscorpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print(len(set(aid)))
print(len(author_pid))
print(len(authors))
print(len(rscorpus))

In [ ]:
print(len(author_corpus))
author_id = author_to_paper[author_to_paper.pos==1]['aid'].values
author_corpus = author_corpus[author_corpus['aid'].isin(author_id)]
author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
print(len(author_corpus))

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_to_paper = author_to_paper.groupby("aid")
author_to_paper_more_2 = author_to_paper.filter(lambda x: len(x) > 2)
author_to_paper_less_2 = author_to_paper.filter(lambda x: len(x) < 3)
author_to_paper_more_2.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
author_to_paper_less_2.to_json(folder_path+'author-2-paper-less.json', orient='records', lines=True)
print("success")

In [ ]:
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
rs_corpus = rs_corpus[~rs_corpus['id'].isin(author_to_paper_less_2['pid'])]
rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus = author_corpus[author_corpus['aid'].isin(author_to_paper['aid'])]
author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus = author_corpus[li]
author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_to_paper_2 = author_to_paper[~author_to_paper['aid'].isin(author_corpus['aid'])]
author_to_paper = author_to_paper[author_to_paper['aid'].isin(author_corpus['aid'])]
author_to_paper.to_json(folder_path+'author-2-paper.json', orient='records', lines=True)
print(len(author_to_paper_2))
print(len(author_to_paper))

In [ ]:
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
rs_corpus = rs_corpus[~rs_corpus['id'].isin(author_to_paper_2['pid'])]
rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)

In [ ]:
print(len(rs_corpus))
print(len(set(author_to_paper['aid'])))

## Reading Data
#### -> junior-authors-more-2.csv

In [ ]:
junior_authors_more_2 = pd.read_csv(folder_path+'junior-authors-more-2.csv', sep=',', header=0, index_col=False)
junior_authors_more_2 = junior_authors_more_2['junior_authors'].sort_values()
print("success")

#### -> junior-authors-less-2.csv

In [ ]:
junior_authors_less_2 = pd.read_csv(folder_path+'junior-authors-less-2.csv', sep=',', header=0, index_col=False)
junior_authors_less_2 = junior_authors_less_2['junior_authors'].sort_values()
print("success")

#### -> junior-authors.csv

In [ ]:
junior_authors = pd.read_csv(folder_path+'junior-authors-corpus.csv', sep=',', header=0, index_col=False)
junior_authors = junior_authors['junior_authors'].sort_values()
print("success")

#### -> author-2-paper.json

In [ ]:
author_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
print("success")
print(len(rs_corpus))

In [ ]:
paper_id = author_paper[author_paper['pid'].isin(junior_authors_less_2)]
print("success")
print(len(paper_id['pid']))

In [ ]:
new_rs_corpus = rs_corpus[~rs_corpus['id'].isin(paper_id['pid'])]
new_rs_corpus.to_json(folder_path+'rs-corpus.json', orient='records', lines=True)
print("success")

## Operation on author corpus
#### -> read author-corpus.json
#### -> delete authors in author-corpus, who have less-2 publications
#### -> update author-corpus with more-2 publications

In [ ]:
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
junior_authors_more_2 = author_corpus[author_corpus['aid'].isin(junior_authors)]
junior_authors_more_2.to_json(folder_path+'author-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
vs_corpus = pd.read_json(folder_path+'vs_corpus_stdev.json', orient='records', encoding='utf8', lines=True)
#grouped_vs_corpus = vs_corpus.groupby("year")
#grouped_vs_corpus.to_json(folder_path+'vs-corpus2.json', orient='records', lines=True)
print("success")

In [ ]:
ids = [v.id for k, v in vs_corpus.iterrows() if len(v.title) < 3]
print(ids)

In [ ]:
vs_corpus = vs_corpus[~vs_corpus.id.isin(ids)]
vs_corpus.to_json(folder_path+'vs_corpus_stdev.json', orient='records', lines=True)

In [ ]:
for index, v in grouped_vs_corpus:
    print(index)
    print(v.count())

### Operations on RS corpus & pre-processed corpus
#### -> sort dataset by year
#### -> save dataset into pre-processed corpus

In [ ]:
#load pre-processed json data again if needed
file_preprocessed = open(folder_path+'vs-preprocessed-corpus.json', 'w+', encoding="utf8")

data_set = {}
def add_doc_to_preprocessed(year, preprocessed_data):
    data_set[str(year)] = preprocessed_data
    data = {}
    data[str(year)] = preprocessed_data
    file_preprocessed.write((json.dumps(data)+'\n'))
    
data_set5 = []
data_set6 = []
data_set7 = []
data_set8 = []
data_set9 = []
count = 0
with open(folder_path+'vs_corpus_avg.json','r', encoding="utf8") as f:
    for doc in f:
        count += 1
        if count % 2500 == 0:
            print(count, "doc processed")
        row = json.loads(doc)
        pub_year = int(row['year'])
        data = {}
        data['id'] = row['id']
        data['year'] = pub_year
        data['title'] = row['title']
        #data['abstract'] = row['abstract']
        if pub_year == 2010:
            data_set5.append(data)
        elif pub_year == 2011:
            data_set6.append(data)
        elif pub_year == 2012:
            data_set7.append(data)
        elif pub_year == 2013:
            data_set8.append(data)
        elif pub_year == 2014:
            data_set9.append(data)


    print("pre-processed corpus loading start")
    add_doc_to_preprocessed(2010, data_set5)
    print('5',len(data_set5))
    add_doc_to_preprocessed(2011, data_set6)
    print('6',len(data_set6))
    add_doc_to_preprocessed(2012, data_set7)
    print('7',len(data_set7))
    add_doc_to_preprocessed(2013, data_set8)
    print('8',len(data_set8))
    add_doc_to_preprocessed(2014, data_set9)
    print('9',len(data_set9))
    file_preprocessed.close()
    print("pre-processed corpus successfully loaded")

### Most frequent terms remove
#### -> select specific year

In [ ]:
#select specific year data for LDA topic modeling, set year = ? e.g. 2005
pub_year = 2009
for year in data_set:
    if int(year) == pub_year:
        doc_year = data_set[str(year)]
        print(year,"corpus successfully selected")
        break
        print(bcolors.STEXT+bcolors.HEADER+"check common tokens, then run next 2 cells"+bcolors.ENDC+bcolors.EETEXT)

#### -> Count terms frequency

In [ ]:
# count tokens to list
count = Counter(word for row in doc_year for word in row['title'])
print("Common term successfully counted...")

#### -> select most common terms

In [ ]:
# select most common terms for removal
#31
num_words = 50
most_common_words = count.most_common(num_words)
#print(bcolors.STEXT+bcolors.HEADER+"remove common tokens"+bcolors.ENDC+bcolors.ETEXT)

if len(most_common_words)>0:
    df = pd.DataFrame(most_common_words, columns = ['Word', 'Count'])
    df.plot.bar(x='Word',y='Count')
    plt.grid(axis = 'y', color ='white', linestyle='-')

for w in range(num_words):
    print(w, most_common_words[w])

# 2005 index 0,1,2,5,6,7,8,10,11,12,13,15,17,21,22,23,24,29,38,39,49
# 2006 index 0,2,3,4,5,6,7,10,11,12,13,14,17,19,23,29,31,34,35,45,47,48,49
# 2007 index 0,1,2,3,5,6,7,8,11,12,13,14,15,16,17,22,23,24,34,37,39,40,41,44,45,49
# 2008 index 0,1,2,3,5,6,7,8,9,10,11,12,15,16,23,26,30,35,36,38,38,39,40,41,42,46,47,48
# 2009 index 0,1,2,3,5,6,7,10,11,12,13,16,23,24,28,29,31,33,37,40,41,43,44,46,47

# 2005 index 0,1,2,3,4,7,8,9,10,11,12,13,15,16,17,18,19,20,21
# 2006 index 0,2,3,4,5,6,10,11,12,13,14,15,16,19,20,21,22,24,25,28,38,31,33,34,35,37
# 2007 index 0,2,3,4,5,6,7,8,11,12,13,14,17,20,21,22,25,26,27,29,30,31,33,34,37,39,40,42,43,44,45,46,47,49
# 2008 index 0,1,3,6,7,9,10,11,13,14,16,19,21,22,23,25,26,30,31,32,35,36,38,39,40,42,45,46
# 2009 index 0,1,4,7,9,10,12,13,17,18,19,20,22,24,25,27,29,30,32,33,34,36,41,42,43

#### -> remove most common terms
#### -> update pre-processed dataset

In [ ]:
# remove common terms form dataset
# please run it carefully, it will remove data
common_index = [0,1,4,7,9,10,12,13,17,18,19,20,22,24,25,27,29,30,32,33,34,36,41,42,43]
print('common_index:',len(common_index))
common_word_list = [ most_common_words[i][0] for i in common_index ]

dummy_text = []
texts = []
# remove most common words
for row in doc_year:
    term_list = [t for t in row['title'] if t not in common_word_list]
    # add year and tokens in list
    data = {}
    data['id'] = row['id']
    data['year'] = row['year']
    data['title'] = term_list
    #data['abstract'] = row['abstract']
    dummy_text.append(data)
    texts.append(term_list)

doc_year.clear()
doc_year = dummy_text
data_set[str(pub_year)] = dummy_text

print("Common terms successfully removed from corpus",pub_year,"...")
#print(bcolors.STEXT+bcolors.HEADER+"save new dataset into sorted file"+bcolors.ENDC+bcolors.ETEXT)

In [ ]:
#store sorted data into file
with open(folder_path+'preprocessed-corpus.json', 'w+', encoding='utf8') as outfile:
    for year in data_set:
        data = {}
        data[str(year)] = data_set[year]
        outfile.write(json.dumps(data)+'\n')
    outfile.close()
    print("pre-processed data has been updated")
#    print(bcolors.STEXT+bcolors.HEADER+"models topics"+bcolors.ENDC+bcolors.ETEXT)

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_to_paper_pid_grouped = author_to_paper.groupby("pid")
unique_author = list(set(author_to_paper['aid']))

In [ ]:
author_score = []
for first_auth in unique_author:
    for second_auth in unique_author:
        if first_auth != second_auth:
            data = {}
            ac_lk = 0.0
            pac_k = 0.0
            for pid, doc in author_to_paper_pid_grouped:
                aid = list(doc['aid'])
                if first_auth in aid and second_auth in aid:
                    ac_lk += (1/int(doc[doc.aid == first_auth]['pos']))
                    ac_lk += (1/int(doc[doc.aid == second_auth]['pos']))
                if second_auth in aid:
                    pac_k += (1/int(doc[doc.aid == second_auth]['pos']))

            data['first_auth'] = first_auth
            data['second_auth'] = second_auth
            data['acw'] = ac_lk/pac_k
            author_score.append(data)
            
pd.DataFrame(author_score).to_json(folder_path+'acw.json', orient='records', lines=True)
print('success')

In [ ]:
author = pd.read_json(folder_path+'HSRank/author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus = pd.read_csv(folder_path+'author-corpus.csv', sep=',', names=['aid', 'author', 'affiliation', 'n_paper', 'h_index', 'n_citation'])
print('success')

In [ ]:
hs_authors = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus = pd.read_csv(folder_path+'author-info.csv', names=['aid', 'n_paper', 'h_index', 'n_citation'])
author_corpus.drop_duplicates(keep='first', inplace=True)
print("success")

In [ ]:
hs_author = []
c = 0
for index, auth in hs_authors.iterrows():
    row = author_corpus[author_corpus.aid == auth.aid]

    if int(auth.aid) == int(row.aid):

        if int(row.n_paper) > int(auth.n_paper):
            auth.n_paper = int(row.n_paper)
            c += 1

        if int(row.h_index) > int(auth.h_index):
            auth.h_index = int(row.h_index)
            c += 1

        if int(row.n_citation) > int(auth.n_citation):
            auth.n_citation = int(row.n_citation)
            c += 1

    hs_author.append(auth)

print(c)
hs_author = pd.DataFrame(hs_author)
hs_author.to_json(folder_path+'hs-author-corpus.json', orient='records', lines=True)
hs_author.head()

In [ ]:
author_info = []
with open(folder_path+'author-corpus.csv', 'r', encoding='utf8') as f:
    for doc in f:
        row = [x.strip() for x in doc.split(',')]
        row_len = len(row)
        author_info.append([row[0], row[row_len-3], row[row_len-2], row[row_len-1]])
    pd.DataFrame(author_info).to_csv(folder_path+'author-info.csv', sep=',', encoding='utf-8', header=None, index=False)

In [ ]:
print(coauthor_corpus[coauthor_corpus.aid.isin(star_author.aid.head(50))].cid.nunique())
print(coauthor_corpus[coauthor_corpus.aid.isin(wmi_author.aid.head(50))].cid.nunique())
print(coauthor_corpus[coauthor_corpus.aid.isin(hs_author.aid.head(50))].cid.nunique())

In [ ]:
import _thread
import time

# Define a function for the thread
def print_time( threadName, delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        print ("%s: %s" % ( threadName, time.ctime(time.time()) ))

# Create two threads as follows
try:
    _thread.start_new_thread( print_time, ("Thread-1", 2, ) )
    _thread.start_new_thread( print_time, ("Thread-2", 4, ) )
except:
    print ("Error: unable to start thread")

while 1:
    pass
    print("success")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(["I'd like an apple", "An apple a day keeps the doctor away", "Never compare an apple to an orange",
                             "I prefer scikit-learn to Orange"])
(tfidf * tfidf.T).A
print (cosine_sim('a little bird', 'a little bird'))
print (cosine_sim('a little bird', 'a little bird chirps'))
print (cosine_sim('a little bird', 'a big dog barks'))

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load('en')
doc1 = nlp('Hello hi there!')
doc2 = nlp('Hello hi there!')
doc3 = nlp('Hey whatsup?')

print(doc1.similarity(doc2)) # 0.999999954642
print(doc2.similarity(doc3))  # 0.699032527716
print(doc1.similarity(doc3))  # 0.699032527716

#### OLD Smimilarity

In [ ]:
#calculate papers and topics sentence similarity score

#store sorted data into file

outfile_paper_similarity = open(folder_path+'vs-topic-paper-similarity-corpus.csv', 'w', newline='')
paper_sim_writer = csv.writer(outfile_paper_similarity, quoting=csv.QUOTE_ALL)

def write_paper_similarity(paper_rows):
    paper_sim_writer.writerows(paper_rows)
    outfile_paper_similarity.flush()
    
def sentence_similarity(synsets1, synsets2, synsets1_technical, synsets2_technical):
    
    paper_score = 1/len(synsets2+synsets2_technical)
    score = 0.0
    
    for itech in synsets2_technical:
        if itech in synsets1_technical:
            score += (paper_score+paper_score)
    
    # For each word in the first sentence
    for synset in synsets2:

        # Get the similarity value of the most similar word in the other sentence
        best_score = [synset.path_similarity(ss) for ss in synsets1 if synset.path_similarity(ss) is not None]

        if len(best_score)>=1:
            if max(best_score)>0.49:
                score += paper_score
                
    return score

def get_topic_similarity(topic):
    #get year only e.g. 2005, 2006, 2007, 2008 and 2009
    topic_year = {}
    for year in syn_data_set:
        topic_score, paper_count = 0.0, 0.0
        clear_output()
        print(topic['id']," : ",year)
        intyear = int(year)
        topic_paper_similarity = []
        for row in syn_data_set[year]:
            
            synsets1 = topic['syn']
            synsets2 = row['syn']
            synsets1_technical = topic['tech']
            synsets2_technical = row['tech']
            
            paper_score = 1/len(synsets2+synsets2_technical)
            score = 0.0

            for itech in synsets2_technical:
                if itech in synsets1_technical:
                    score += (paper_score+paper_score)

            # For each word in the first sentence
            for synset in synsets2:

                # Get the similarity value of the most similar word in the other sentence
                best_score = [synset.path_similarity(ss) for ss in synsets1 if synset.path_similarity(ss) is not None]

                if len(best_score)>=1:
                    if max(best_score)>0.49:
                        score += paper_score
            
            #sim_score = sentence_similarity(topic['syn'], row['syn'], topic['tech'], row['tech'])
            data = []
            data.append(topic['id'])
            data.append(row['id'])
            data.append(intyear)
            data.append(score)
            topic_paper_similarity.append(data)
            #write_paper_similarity(data)
        #_thread.start_new_thread(write_paper_similarity, (topic_paper_similarity,))
        paper_sim_writer.writerows(topic_paper_similarity)
        outfile_paper_similarity.flush()


if __name__ == "__main__":
    
    #pool = ThreadPool(16)
    print("similarity preprocessing has been started...")
    #pool.starmap(get_topic_similarity, zip(syn_lda_topic))
    for topic in syn_lda_topic:
        _thread.start_new_thread( get_topic_similarity, (topic, ) )
    #pool.close() 
    #pool.join()
    print("similarity has been successfully measured and saved")
    print(bcolors.STEXT+bcolors.HEADER+"reload sorted dataset"+bcolors.ENDC+bcolors.ETEXT)

In [ ]:
def penn_to_wn(tag):
    if tag.startswith('N'):
        return 'n'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('J'):
        return 'a'
    elif tag.startswith('R'):
        return 'r'
    else:
        return None
    
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    #elif wn_tag is 'v':
        #return ''
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

def get_sentence_synset(doc):
    # Tokenize and tag
    sentence = pos_tag(doc[topic_selection])
    
    # Get the synsets for the tagged words
    synsets = [tagged_to_synset(*tagged_word) for tagged_word in sentence]
    
    synsets_index = [i for i in range(len(synsets)) if synsets[i] is None]
    synsets_technical = [sentence[ip][0] for ip in synsets_index]
    
    # Filter out the Nones
    synsets = [ss for ss in synsets if ss]
    
    data = {}
    data['id'] = doc['id']
    data['tech'] = synsets_technical
    data['syn'] = synsets
    return data

syn_lda_topic = []
syn_data_set = {}
if __name__ == "__main__":
    
    pool = ThreadPool(16)
    print("Synset preprocessing has been started...")
    syn_lda_topic = pool.starmap(get_sentence_synset, zip(lda_topics))
    print('Topics completed...')
    for year in data_set:
        syn_data_set[str(year)] = pool.starmap(get_sentence_synset, zip(data_set[str(year)]))
        print(year, 'completed...')
    pool.close() 
    pool.join()
    print("Synset has been successfully measured and saved...")

In [ ]:
data_set_length = {}
with open(folder_path+'vs-preprocessed-corpus.json', 'r', encoding='utf8') as f:
    for doc in f:
        data = json.loads(doc)
        for year in data:
            data_set_length[str(year)] = len(data[year])
    f.close()
    print ("data set length successfully loaded...")
    
df_topic_similarity = pd.read_csv(folder_path+'vs-topic-paper-similarity-corpus.csv', names=['tid', 'pid', 'year', 'sim'])
grouped_topic_similarity = df_topic_similarity.groupby(['tid'])
print("success")

In [ ]:
#Read LDA topics file if needs
lda_topics = []
with open(folder_path+'lda-title-topics_lst.json','r', encoding='utf8') as f:
    for doc in f:
        topic = json.loads(doc)
        topic['title'] = nlp(topic['title'])
        lda_topics.append(topic)
    f.close()
print(len(lda_topics), "Topics successfully readed...")

In [ ]:
vs_corpus_nlp = []
def getNLP(i, row):
    
    row.title = nlp(row.title)
    vs_corpus_nlp.append(row)
    clear_output()
    print(i)
    
vs_corpus = pd.read_json(folder_path+'vs_corpus_avg.json', orient='records', encoding='utf8', lines=True)    
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    {executor.submit(getNLP, i, row): row for i, row in vs_corpus.iterrows()}
    
print(len(vs_corpus_nlp), "success")

In [ ]:
#nlp = spacy.load('en')
outfile_paper_similarity = open(folder_path+'vs-topic-paper-similarity-corpus.csv', 'w', newline='')
paper_sim_writer = csv.writer(outfile_paper_similarity, quoting=csv.QUOTE_ALL)

def write_paper_similarity(paper_rows):
    print("saving start")
    paper_sim_writer.writerows(paper_rows)
    outfile_paper_similarity.flush()
    print("saving end")

def get_topic_similarity(topic):
    
    c = 0
    topic_paper_similarity = []
    for row in vs_corpus_nlp:
        
        topic_paper_similarity.append([topic['id'], row['id'], row['year'], (topic['title'].similarity(row['title']))])
        c += 1
        clear_output()
        print(c)

        if c > 4999:
            #write_paper_similarity(topic_paper_similarity)
            threading.Thread(target=write_paper_similarity, args=(topic_paper_similarity,)).start() 
            topic_paper_similarity = []
            c = 0

    #write_paper_similarity(topic_paper_similarity)
    threading.Thread(target=write_paper_similarity, args=(topic_paper_similarity,)).start() 
    
if __name__ == "__main__":
    
    pool = ThreadPool(4)
    print("similarity preprocessing has been started...")
    pool.starmap(get_topic_similarity, zip(lda_topics))
    pool.close()
    pool.join()
    print("success")

In [ ]:
uncomplete_similarity_lst = pd.read_csv(folder_path+'complete_similarity.csv', names=['tid', 'pid', 'year', 'sim'])
uncomplete_similarity_lst = uncomplete_similarity_lst.groupby(['tid'])
topic_id = uncomplete_similarity_lst.groups.keys()
print("success")

In [ ]:
#nlp = spacy.load('en')
outfile_paper_similarity = open(folder_path+'vs-topic-paper-similarity-corpus.csv', 'w', newline='')
paper_sim_writer = csv.writer(outfile_paper_similarity, quoting=csv.QUOTE_ALL)

def write_paper_ucsimilarity(paper_rows):
    print("saving start")
    paper_sim_writer.writerows(paper_rows)
    outfile_paper_similarity.flush()
    print("saving end")

def get_topic_ucsimilarity(tid):
    
    c = 0
    topic_paper_similarity = []
    for topic in lda_topics:
        if topic['id'] == tid:
            for row in vs_corpus_nlp:

                topic_paper_similarity.append([topic['id'], row['id'], row['year'], (topic['title'].similarity(row['title']))])
                c += 1
                clear_output()
                print(c)

                if c > 4999:
                    threading.Thread(target=write_paper_ucsimilarity, args=(topic_paper_similarity,)).start() 
                    topic_paper_similarity = []
                    c = 0

            threading.Thread(target=write_paper_ucsimilarity, args=(topic_paper_similarity,)).start()
            break
    
if __name__ == "__main__":
    
    pool = ThreadPool(4)
    print("similarity preprocessing has been started...")
    pool.starmap(get_topic_similarity, zip(topic_id))
    pool.close()
    pool.join()
    print("success")

In [ ]:
lda_topics = pd.DataFrame(lda_topics)
lda_topics.head()

pd.DataFrame(str(complete)).to_json(folder_path+'complete_topics2.json', orient='records', lines=True)
uncomplete.to_json(folder_path+'uncomplete_topics3.json', orient='records', lines=True)

complete = lda_topics[lda_topics.id.isin(complete_topic)].tid
uncomplete = lda_topics[lda_topics.id.isin(df_topic['tid'])].tid
print("success")

df_topic_similarity = pd.read_csv(folder_path+'complete_similarity.csv', names=['tid', 'pid', 'year', 'sim'])
#df_topic_similarity.drop_duplicates(keep='first', inplace=True)
#grouped_topic_similarity = df_topic_similarity.groupby(['tid'])
print("success")

df_topic_similarity.drop_duplicates(subset=['tid', 'pid'], keep='first', inplace=True)
grouped_topic_similarity = df_topic_similarity.groupby(['tid'])
print("success")